In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## close range vrs distance range relative to the trollinde radar.

In [2]:
track = pd.read_csv('../saved-files/threshold-15/all-cells-first_landfall-threshold-15.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,293,13,192.616195,391.192565,139,15,22775,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.252616e+06,...,69.527152,17.790576,6538,0 days 00:00:00,76.951744,15.036292,38.598690,176.101887,True,0
2045,294,30,274.392543,289.651057,4,15,22833,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,68.889895,15.058896,6549,0 days 00:00:00,54.214501,15.196207,39.099655,6.965209,False,5
2046,295,33,274.600400,291.595498,7,15,22877,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.334600e+06,...,68.886437,15.105794,6549,0 days 00:05:00,53.226607,15.270663,40.109338,10.944861,True,0
2047,296,33,285.786159,372.620721,74,15,22919,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.345786e+06,...,68.706795,17.038362,6569,0 days 00:00:00,72.446871,15.390894,40.106802,133.337769,True,0


### Calculate distance relative to trollinde radar

In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[23.538328571313603,
 26.876486946141238,
 30.868929748544517,
 34.92463663702369,
 38.59792014091538,
 76.11243007337428,
 108.53886934691211,
 125.3997527447183,
 150.60133114486698,
 30.261435088683257]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 751 µs, sys: 8 µs, total: 759 µs
Wall time: 765 µs


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0


# close domain

In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_15 = track[ mask_lat]
close_domain_15

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,292,28,244.395871,311.000000,2,15,22745,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.304396e+06,...,69.144022,15.650531,6534,0 days 00:00:00,17.643001,14.985573,38.596077,46.765506,False,5
2042,293,27,246.587405,313.122095,5,15,22789,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306587e+06,...,69.122231,15.697394,6534,0 days 00:05:00,17.932251,15.036292,38.598690,32.834525,True,0
2043,292,34,270.000000,314.720218,3,15,22751,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908386,15.679432,6536,0 days 00:00:00,39.217544,14.985573,38.596077,15.919946,True,0
2045,294,30,274.392543,289.651057,4,15,22833,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,68.889895,15.058896,6549,0 days 00:00:00,54.214501,15.196207,39.099655,6.965209,False,5


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_15['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_15[close_domain_15['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 128 ms, sys: 1.75 ms, total: 129 ms
Wall time: 158 ms


In [9]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_15['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_43664/3053280065.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_15['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values


In [10]:
close_domain_15 = close_domain_15.reset_index(drop=True)

In [11]:
close_domain_15.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-close
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0,0


In [12]:
len(close_domain_15)

528

# distant domain

In [13]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_15 = track[ mask_lat ]
distant_domain_15

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,292,28,244.395871,311.000000,2,15,22745,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.304396e+06,...,69.144022,15.650531,6534,0 days 00:00:00,17.643001,14.985573,38.596077,46.765506,False,5
2042,293,27,246.587405,313.122095,5,15,22789,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306587e+06,...,69.122231,15.697394,6534,0 days 00:05:00,17.932251,15.036292,38.598690,32.834525,True,0
2043,292,34,270.000000,314.720218,3,15,22751,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908386,15.679432,6536,0 days 00:00:00,39.217544,14.985573,38.596077,15.919946,True,0
2045,294,30,274.392543,289.651057,4,15,22833,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,68.889895,15.058896,6549,0 days 00:00:00,54.214501,15.196207,39.099655,6.965209,False,5


In [14]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_15['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_15[distant_domain_15['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 372 ms, sys: 0 ns, total: 372 ms
Wall time: 375 ms


In [15]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_15['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_43664/3882188696.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_15['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values


In [16]:
distant_domain_15 = distant_domain_15.reset_index(drop=True)


In [17]:
distant_domain_15.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-distant
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,...,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False,20,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,...,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False,15,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False,10,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False,5,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True,0,0


In [18]:
len(distant_domain_15)

1236

In [19]:
len(close_domain_15)

528

In [20]:
len(track)

2049

In [21]:
len(close_domain_15) + len(distant_domain_15)

1764

In [22]:
distant_domain_15['latitude'].max(), distant_domain_15['latitude'].min(),close_domain_15['latitude'].max(), close_domain_15['latitude'].min()

(70.59571518347992, 67.89196414054493, 69.68355786048062, 68.78919651359978)

In [23]:
distant_domain_15['longitude'].max()

15.999723092447566

In [24]:
close_domain_15.to_csv('../saved-files/threshold-15/close-domain-threshold-15-LF.csv', index=False)
distant_domain_15.to_csv('../saved-files/threshold-15/distant-domain-threshold-15-LF.csv', index=False)

# END

In [42]:
# max_dBZ_10 = []
# for indexes, time in enumerate(np.unique(close_domain_10['relative-time-to-radar'])):
#     # Sorting based on same relative times
#     sort_relative_times_threshold_10 = close_domain_10[close_domain_10['relative-time-to-radar'] == time]
#     # Find each relative time associated with a particular reflectivity value
    
#     max_dbZ_threshold_10 = np.mean(sort_relative_times_threshold_10['max dBZ'])
#     max_dBZ_10.append(max_dbZ_threshold_10)